In [1]:
import os
import yaml
import urllib
from PIL import Image
from enum import Enum
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import xml.etree.cElementTree as ET
import glob
import argparse
import numpy as np
import json
import numpy
import cv2
from collections import OrderedDict
import scipy.misc
from skimage import measure   
from shapely.geometry import Polygon, MultiPolygon, MultiPoint
import random
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import shutil
import pickle

WORKING_DIRECTORY = '/media/dean/datastore/datasets/darknet_evaluate'
COCO_DIRECTORY = os.path.join(WORKING_DIRECTORY, 'data', 'coco')
BDD10K_COCO_ANNOTATIONS_FILE = os.path.join(COCO_DIRECTORY,'annotations' , 'bdd10k_instances_train2014.json')
IMAGES_DIRECTORY = os.path.join(COCO_DIRECTORY, 'images','train2014')
WEIGHTS_DIRECTORY = os.path.join(WORKING_DIRECTORY,'trained_weights')

In [2]:
bdd10k_coco = COCO(BDD10K_COCO_ANNOTATIONS_FILE)

loading annotations into memory...
Done (t=0.80s)
creating index...
index created!


In [3]:
ann_ids = bdd10k_coco.getAnnIds()
print(len(ann_ids))

112110


In [4]:
#!java -jar $WORKING_DIRECTORY/data/cocotoyolo.jar "{BDD10K_COCO_ANNOTATIONS_FILE}" "{IMAGES_DIRECTORY}/" "all" "{COCO_DIRECTORY}/labels/train2014"

In [ ]:
## Get All weights files and sort
d = {}
weights_files = glob.glob(os.path.join(WEIGHTS_DIRECTORY, '*.weights'))

# Grab epochs
for weights in weights_files:
    fname = os.path.split(weights)[-1]
    epochs = fname[15:].split('.weights')[0]
    d[fname] = int(epochs)
    
    
sorted_weights = OrderedDict(sorted(d.items(), key=lambda kv: kv[1]))
bdd_weights = OrderedDict()

for weights_file in sorted_weights.keys():
    d = {}
    d['epochs'] = sorted_weights[weights_file]
    d['map_results_file'] = os.path.join(WEIGHTS_DIRECTORY, weights_file+'.txt')
    
    if not os.path.exists(d['map_results_file']):
        outfile = !cd $WORKING_DIRECTORY && ./darknet detector map cfg/bdd100k.data cfg/yolov3-bdd100k.cfg $WEIGHTS_DIRECTORY/$weights_file
        with open(d['map_results_file'],"w+") as f:
            f.write(outfile)    
    
    #Get MAP results from file
    class_stats = []
    map_stats = {}
    with open(d['map_results_file']) as openfile:
        for line in openfile:
            for part in line.split():
                if 'class_id' in part:
                    class_map = {}
                    tokens = line.split()
                    class_map['class_id'] = int(tokens[2].strip(','))
                    class_map['class_name'] = tokens[5].strip(',')
                    tokens = line.split('ap = ')
                    class_map['class_ap'] = tokens[1].split()[0].strip(',')

                    class_stats.append(class_map)
                elif '(mAP)' in part:
                    tokens = line.split('(mAP) =')[1]
                    map_stats['mean_avg_precision'] = tokens.split()[2].strip(',')
            if 'Total Detection Time:' in line:
                tokens = line.split(':')[1]
                total_detection_time = float(tokens.split()[0])
                map_stats['total_detection_time'] = total_detection_time
    map_stats['class_stats'] = class_stats
    print(map_stats) 
    bdd_weights[weights] = map_stats

    
    
# Make sure Dictionary is made properly
print(bdd_weights)


In [ ]:
class_stats = []
map_stats = {}
with open("/media/dean/datastore/datasets/darknet_evaluate/yolov3-bdd100k_24864.weights.txt") as openfile:
    for line in openfile:
        for part in line.split():
            if 'class_id' in part:
                class_map = {}
                tokens = line.split()
                class_map['class_id'] = int(tokens[2].strip(','))
                class_map['class_name'] = tokens[5].strip(',')
                tokens = line.split('ap = ')
                class_map['class_ap'] = tokens[1].split()[0].strip(',')
                
                class_stats.append(class_map)
            elif '(mAP)' in part:
                tokens = line.split('(mAP) =')[1]
                map_stats['mean_avg_precision'] = tokens.split()[2].strip(',')
        if 'Total Detection Time:' in line:
            tokens = line.split(':')[1]
            total_detection_time = float(tokens.split()[0])
            map_stats['total_detection_time'] = total_detection_time
map_stats['class_stats'] = class_stats
print(map_stats)         


In [ ]:
# #initialize COCO ground truth api
# cocoGt=COCO(BDD10K_COCO_ANNOTATIONS_FILE)

# bdd10k_coco_results = []

# for results_file in bdd10k_coco_results:
#     #initialize COCO detections api
#     resFile='/home/dean/Desktop/coco_results.json'
#     cocoDt=cocoGt.loadRes(resFile)

#     for cat in category_names:
#         category_ids = cocoDt.getCatIds(catNms=cat)
#         imgIds=cocoDt.getImgIds(catIds=category_ids)
#         print('MAP for:', cat)
#         # running evaluation
#         cocoEval = COCOeval(cocoGt,cocoDt,annType)
#         cocoEval.params.imgIds  = imgIds
#         cocoEval.evaluate()
#         cocoEval.accumulate()
#         cocoEval.summarize()
#         print('\n\n\n')